In [1]:
import pandas as pd
import requests
from pathlib import Path
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Load csv with all the information about the arts

In [2]:
artworks_csv = pd.read_csv("Tate_artwork_data.csv")
artworks_csv.columns



/home/michal/miniconda3/envs/SAR/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['id', 'accession_number', 'artist', 'artistRole', 'artistId', 'title',
       'dateText', 'medium', 'creditLine', 'year', 'acquisitionYear',
       'dimensions', 'width', 'height', 'depth', 'units', 'inscription',
       'thumbnailCopyright', 'thumbnailUrl', 'url'],
      dtype='object')

In [3]:
artworks_csv = artworks_csv[artworks_csv.thumbnailUrl.notnull()]

## Display the art categories and picking the right ones

In [4]:
artworks_csv.medium.unique()

array(['Watercolour, ink, chalk and graphite on paper. Verso: graphite on paper',
       'Graphite on paper', 'Graphite on paper. Verso: graphite on paper',
       ...,
       'Film, 16 mm, 2 projections, black and white, and sound (stereo) and smoke machine',
       'Wood, Perspex, plaster, wire, tampons and human blood',
       'Wood, Perspex, clock case, tampons and human blood'], dtype=object)

In [5]:
wanted_mediums = ['Screenprint on paper']

In [6]:
artworks_filtered = artworks_csv.loc[artworks_csv.medium.isin(wanted_mediums)]
artworks_filtered.medium.unique()

array(['Screenprint on paper'], dtype=object)

## Downloading the images

In [7]:
artworks_filtered.head()
len(artworks_filtered)

2564

In [8]:
def save_image_from_response(dest_dir, filename, response):
    
    filename = filename.replace("/","-")
    
    path = Path.cwd() / dest_dir / filename
        
    if response.status_code == 200:
        with open(path, 'wb') as f:
            f.write(response.content)

            

In [9]:
def download_images(artworks, dir_name, sample_size = -1, generic_name = False):

    # create the directory where the downloaded images will be stored
    dir_path = Path.cwd() / dir_name
    dir_path.mkdir(parents=True, exist_ok=True)
    
    # take N random rows from the table
    if sample_size != -1:
        artworks_csv_sample = artworks_filtered.sample(sample_size)
    else:
        artworks_csv_sample = artworks_filtered
    counter = 0
    for index, row in artworks_csv_sample.iterrows():
        
        art_title = row.title
        filename = art_title + ".jpg" if not generic_name else "image_" + str(index) + ".jpg"

        # For full resolution I need to modify the URL a bit, it follows a strict pattern
        imageURL = row.thumbnailUrl
        imageURL = imageURL.replace("_8.jpg", "_10.jpg")

        response = requests.get(imageURL, verify = False)
        
        if counter % 100 == 0:
            print(f"Still downloading...")
            print(f"Images remaining: {len(artworks_csv_sample)-counter}")
        
        save_image_from_response(dir_name, filename, response)
        counter += 1
    
    print("Download finished.")

In [ ]:
download_to = "Tate-all-screenprint-paper"
download_images(artworks_filtered, download_to, generic_name = True)


Still downloading...
Images remaining: 2564
Still downloading...
Images remaining: 2464
Still downloading...
Images remaining: 2364
Still downloading...
Images remaining: 2264
Still downloading...
Images remaining: 2164
Still downloading...
Images remaining: 2064
Still downloading...
Images remaining: 1964
Still downloading...
Images remaining: 1864
Still downloading...
Images remaining: 1764
Still downloading...
Images remaining: 1664
Still downloading...
Images remaining: 1564
Still downloading...
Images remaining: 1464
Still downloading...
Images remaining: 1364
Still downloading...
Images remaining: 1264
Still downloading...
Images remaining: 1164
Still downloading...
Images remaining: 1064
Still downloading...
Images remaining: 964
Still downloading...
Images remaining: 864
Still downloading...
Images remaining: 764
Still downloading...
Images remaining: 664
Still downloading...
Images remaining: 564
Still downloading...
Images remaining: 464
Still downloading...
Images remaining:

In [ ]:
pd.options.display.max_rows = 4000
artworks_csv['medium'].value_counts()